# New p2n API

## Example I

### Pop2net only

In [7]:
import pop2net as p2n

env = p2n.Environment()

location1 = p2n.Location()
env.add_location(location1)

actor1 = p2n.Actor()
actor2 = p2n.Actor()
env.add_actors([actor1, actor2])

location1.add_actors([actor1, actor2])

### Pop2net + Agentpy

In [8]:
import agentpy as ap

class AgentpyActor(p2n.Actor, ap.Agent):
    def say_hello(self):
        print("hello")

class AgentpyLocation(p2n.Location, ap.Agent):
    def say_goodbye(self):
        print("goodbye")

class AgentpyModel(ap.Model):
    def setup(self):
        self.env = p2n.Environment(model=self, framework="agentpy")
        
        location1 = AgentpyLocation(model=self)
        location1.test123 = True
        self.env.add_location(location1)

        location2 = AgentpyLocation(model=self)
        location2.test123 = False
        self.env.add_location(location2)

        actor1 = AgentpyActor(model=self)
        actor2 = AgentpyActor(model=self)
        self.env.add_actors([actor1, actor2])
        print(self.env.actors)

        location1.add_actors([actor1, actor2])
    
    def step(self):
        self.env.actors.say_hello() # env enables agentlist usage
        self.env.locations.say_goodbye()

model = AgentpyModel()
model.run(steps=10)
    

AgentList (2 objects)
hello
hello
goodbye
goodbye
Completed: 1 stepshello
hello
goodbye
goodbye
Completed: 2 stepshello
hello
goodbye
goodbye
Completed: 3 stepshello
hello
goodbye
goodbye
Completed: 4 stepshello
hello
goodbye
goodbye
Completed: 5 stepshello
hello
goodbye
goodbye
Completed: 6 stepshello
hello
goodbye
goodbye
Completed: 7 stepshello
hello
goodbye
goodbye
Completed: 8 stepshello
hello
goodbye
goodbye
Completed: 9 stepshello
hello
goodbye
goodbye
Completed: 10 steps
Run time: 0:00:00.000654
Simulation finished


DataDict {
'info': Dictionary with 9 keys
'reporters': DataFrame with 1 variable and 1 row
}

### Pop2net + Mesa

In [9]:
import mesa

class MesaActor(p2n.Actor, mesa.Agent):
    def say_hello(self):
        print("hello")

class MesaLocation(p2n.Location, mesa.Agent):
    def say_goodbye(self):
        print("goodbye")

class MesaModel(mesa.Model):
    def __init__(self):
        super().__init__()
        
        self.env = p2n.Environment(model=self, framework="mesa")
        
        location1 = MesaLocation(model=self)
        self.env.add_location(location1)

        actor1 = MesaActor(model=self)
        actor2 = MesaActor(model=self)
        self.env.add_actors([actor1, actor2])

        location1.add_actors([actor1, actor2])
    
    def step(self):
        self.env.actors.do("say_hello") # env enables mesa's AgentSet usage
        self.env.locations.do("say_goodbye")

model = MesaModel()

for i in range(10):
    model.step()

hello
hello
goodbye
hello
hello
goodbye
hello
hello
goodbye
hello
hello
goodbye
hello
hello
goodbye
hello
hello
goodbye
hello
hello
goodbye
hello
hello
goodbye
hello
hello
goodbye
hello
hello
goodbye


## Example II

### No framework

In [10]:
import pop2net as p2n
import random

class OpinionActor(p2n.Actor):
    def __init__(self, opinion):
        super().__init__()
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = [actor for actor in self.env.actors if actor.opinion != self.opinion]
        self.connect(random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, actor):
        return actor.opinion

class Model:
    def __init__(self):
        self.env = p2n.Environment()
        self.env.add_actors([OpinionActor(opinion = i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        for actor in self.env.actors:
            actor.add_connection_to_other_cluster()
    

model = Model()
for _ in range(3):
    model.step()
model.env.inspector.plot_actor_network()


### Pop2net + Agentpy

In [11]:
import pop2net as p2n
import agentpy as ap

class OpinionAgent(p2n.Actor, ap.Agent):
    def setup(self, opinion):
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = self.env.actors.select(self.env.actors.opinion != self.env.actors.opinion) # we can use ap.agentLists now!
        self.connect(self.model.random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, actor):
        return actor.opinion

class Model(ap.Model):
    def setup(self):
        self.env = p2n.Environment(model=self, framework="agentpy")
        self.env.add_actors([OpinionAgent(model=self, opinion=i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        self.env.actors.add_connection_to_other_cluster() # we can use ap.agentLists now!

model = Model()
model.run(steps=3)
model.env.inspector.plot_actor_network()

Completed: 3 steps
Run time: 0:00:00.111789
Simulation finished


### Pop2net + Mesa

In [12]:
import pop2net as p2n
import mesa

class OpinionAgent(p2n.Actor, mesa.Agent):
    def __init__(self, opinion, model):
        super().__init__(model)
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = [actor for actor in self.env.actors if actor.opinion != self.opinion]
        self.connect(self.model.random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, actor):
        return actor.opinion

class Model(mesa.Model):
    def __init__(self, *args, seed = None, rng = None, **kwargs):
        super().__init__(*args, seed=seed, rng=rng, **kwargs)
        self.env = p2n.Environment(model=self, framework="mesa")
        self.env.add_actors([OpinionAgent(model=self, opinion=i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        self.env.actors.do("add_connection_to_other_cluster")

model = Model()
for _ in range(3):
    model.step()
model.env.inspector.plot_actor_network()